Привет, меня зовут Люман. Сегодня я проверю твой проект.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу, видно, что приложено много усилий!
* Работа получилась отличная, можно сказать идеальная!
* Радует, что проект хорошо структурирован. Приятно проверять такие работы.
* Также понравилось составление выводов, их приятно читать!
* Проект зачтен. Удачи в дальнейшем обучении и будущих заданиях.


# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# План работы:

1) [Изучить общую информацию.](#id_1)

2) [Создать 3 выборки (обучающую, валидационную и тестовую).](#id_2)

3) [Настроить гиперпараметры.](#id_3)

4) [Проверить качество моделей.](#id_4)

5) [Проверить модели на адекватность.](#id_5)

6) [Общий вывод.](#id_6)

<a id='id_1'></a>
## 1. Откройте и изучите файл

In [1]:
# импортируем библиотеки, ознакамливаемся с данными
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод по пункту 1:

Всего у нас 5 колонок и 3214 строк.
Пропусков нет, все данные обработаны.

Колонки 'calls', 'minutes', 'messages', 'mb_user' - являются __признаками__, по которым модель будет обучаться.
Колонка 'is_ultra' - __целевой признак__.

<div class="alert alert-block alert-success">
<b>Успех:</b> Первый шаг сделан хорошо, все требуемые методы использованы.
</div>

<a id='id_2'></a>
## 2. Разбейте данные на выборки

In [2]:
# Создадим обучающую, валидационную и тестовые выборки (соотношение сделаем 3:1:1)
# Для начала отделим обучающую выборку (это 60% об общих данных)
from sklearn.model_selection import train_test_split
data_train, data_another = train_test_split(data, test_size=0.4, random_state=123)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2669 to 1346
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [3]:
data_another.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1286 entries, 2377 to 2572
Data columns (total 5 columns):
calls       1286 non-null float64
minutes     1286 non-null float64
messages    1286 non-null float64
mb_used     1286 non-null float64
is_ultra    1286 non-null int64
dtypes: float64(4), int64(1)
memory usage: 60.3 KB


___Тренировочная выборка___ сохранена как data_train, оставшиеся данные (40% от общего числа) поделим на две одинаковые выборки, которые и будут являться валидационной (data_valid) и тестовой (data_test).

In [4]:
data_valid, data_test = train_test_split (data_another, test_size=0.5, random_state=123)
data_valid.info(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1807 to 529
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [5]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2619 to 221
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [6]:
# Для последующего обучения моделей - создадим переменные для признаков и целевого признака для каждой из выборок
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

### Вывод по пункту 2:
Итого у нас получилось 3 выборки:

    1) data_train - обучающая выборка (60% от общих данных)
    
    2) data_valid - валидационная выборка (20% от общих данных)
    
    3) data_test - тестовая выборка (20% от общих данных)
    
У каждой из выборок были созданы переменные для признаков (features) и для целевого признака (target).

<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение сделано абсолютно верно. Молодец, что проанализировал получившиеся наборы.
</div>

<a id='id_3'></a>
## 3. Исследуйте модели

В данной задаче мы оцениваем категориальный признак ('is_ultra' либо 0 либо 1). А значит перед нами задача классификации.
Если бы мы оценивали примерную стоимость платы за тариф, тогда перед нами была бы задача регрессии.

На данном этапе мы проверим 3 модели:

    1) Дерево решений
    
    2) Случайный лес
    
    3) Логистическая регрессия

### 3.1 Дерево решений

In [7]:
# импортируем необходимые библиотеки
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [8]:
# Попробуем поменять гиперпараметры (начнем с max_depth, попробуем его в цикле от 1 до 10)
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(depth, "-", accuracy)

1 - 0.7620528771384136
2 - 0.7900466562986003
3 - 0.80248833592535
4 - 0.8040435458786936
5 - 0.8227060653188181
6 - 0.8149300155520995
7 - 0.8118195956454122
8 - 0.80248833592535
9 - 0.8180404354587869
10 - 0.80248833592535


Лучше всего модель работает при показатели max_depth = 5, а значит все что до 5 отнесем к разряду недообученой модели, все что больше - переобученной модели.

In [9]:
# теперь найдем самый подходящий нам гиперпараметр min_samples_split 
for split in range(2, 11):
    model=DecisionTreeClassifier(max_depth=5, min_samples_split=split, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(split, '-', accuracy)

2 - 0.8227060653188181
3 - 0.8227060653188181
4 - 0.8227060653188181
5 - 0.8227060653188181
6 - 0.8227060653188181
7 - 0.8211508553654744
8 - 0.8211508553654744
9 - 0.8211508553654744
10 - 0.8211508553654744


При минимальном количестве примеров от 2 до 6 - уровень accuracy - не меняется, дальше же он падает, поэтому парметр оставляем на уровне 2.

In [10]:
# перейдем к гиперпараметру min_samples_leaf
for leaf in range (1, 11):
    model=DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=leaf, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(leaf, '-', accuracy)

1 - 0.8227060653188181
2 - 0.8227060653188181
3 - 0.8242612752721618
4 - 0.8227060653188181
5 - 0.8227060653188181
6 - 0.8242612752721618
7 - 0.8227060653188181
8 - 0.8227060653188181
9 - 0.8227060653188181
10 - 0.8227060653188181


Самый высокий показатель accuracy - при гипермараметре min_samples_leaf равном 3 или 6.

In [11]:
# Все необходимые гиперпараметры мы выбрали - время создать модель
tree_model = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=3, random_state=123)
tree_model.fit(features_train, target_train)
# Как было выявлено ранее, accuracy на данной модели = 0.8242612752721618, что примерно около 83%

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [12]:
# Проверим значение accuracy на созданой модели
tree_predict=tree_model.predict(features_valid)
accuracy_score(target_valid, tree_predict)

0.8242612752721618

### 3.2 Случайный лес

In [13]:
# загружаем библиотеку
from sklearn.ensemble import RandomForestClassifier
# по аналогии с моделью "Дерево решений" ищем наиболее подходящие гиперпараметры
for depth in range(1, 11):
    model = RandomForestClassifier(max_depth=depth, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(depth, "-", accuracy)

1 - 0.7589424572317263
2 - 0.7900466562986003


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

3 - 0.7962674961119751
4 - 0.8040435458786936
5 - 0.8102643856920684
6 - 0.8164852255054432
7 - 0.8180404354587869
8 - 0.8195956454121306
9 - 0.8211508553654744
10 - 0.8118195956454122


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
# самый большой уровень accuracy при max_depth = 9, его и оставляем
# далее подбираем min_samples_split
for split in range(2, 11):
    model=RandomForestClassifier(max_depth=9, min_samples_split=split, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(split, '-', accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2 - 0.8211508553654744
3 - 0.8195956454121306


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4 - 0.8227060653188181
5 - 0.8258164852255054


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


6 - 0.8227060653188181
7 - 0.8211508553654744
8 - 0.8211508553654744


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9 - 0.8289269051321928
10 - 0.8320373250388803


In [15]:
# самый лучший показатель - 10, его и оставляем
# далее следует min_samples_leaf
for leaf in range (1, 11):
    model=RandomForestClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=leaf, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(leaf, '-', accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1 - 0.8320373250388803
2 - 0.8320373250388803


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


3 - 0.8273716951788491
4 - 0.8242612752721618


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


- 0.8211508553654744
8 - 0.8195956454121306


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9 - 0.8195956454121306
10 - 0.8195956454121306


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [16]:
# при настройке параметра min_samples_leaf - самый высокий показатель при значении 1, его и оставляем
# теперь посмотрим гиперпараметр n_estimators
for est in range (1, 11):
    model=RandomForestClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=1, n_estimators=est, random_state=123)
    model.fit(features_train, target_train)
    predict=model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(est, '-', accuracy)

1 - 0.7869362363919129
2 - 0.807153965785381
3 - 0.8180404354587869
4 - 0.8133748055987559
5 - 0.8211508553654744
6 - 0.8320373250388803
7 - 0.8273716951788491
8 - 0.8304821150855366
9 - 0.8320373250388803
10 - 0.8320373250388803


Самое высокое значение при уровне - 6, его оставляем.
Параметры мы выяснили, время создать модель

In [17]:
forest_model = RandomForestClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=1, n_estimators=6, random_state=123)
forest_model.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=6,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [18]:
# Проверим accuracy на созданной модели
forest_predict=forest_model.predict(features_valid)
accuracy_score(target_valid, forest_predict)

0.8320373250388803

### 3.3 Логистическая регрессия

In [19]:
# загрузим библиотеку 
from sklearn.linear_model import LogisticRegression

In [20]:
# как известно - логистическая регрессия менее склонная к обучению, чем Случайный лес или Дерево решений
# а значит просто сохраним модель и проверим её аккуратность на валидационной выборке
logistic_model=LogisticRegression(random_state=123)
logistic_model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Проверим accuracy на данной модели 
logistic_predict=logistic_model.predict(features_valid)
accuracy_score(target_valid, logistic_predict)

0.7200622083981337

### Вывод по пункту 3:

Нами были созданы 3 основных модели с максимально подходящими параметрами, accuracy у данных моделей составило:

    - Дерево решений 0.8242612752721618
    
    - Случайный лес 0.8320373250388803
    
    - Логистическая регрессия 0.7200622083981337
    
Лучше всех нам подходит модель "Случайный лес", модель "Дерево решений" отстало всего на 1% правильных ответов, в то время как модель "Логистическая регрессия" проигрывает в качестве (правильных ответов на 10% меньше).

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что для подбора параметров использована только валидационная выборка! Выводы абсолютно верные. 
</div>

<a id='id_4'></a>
## 4. Проверьте модель на тестовой выборке

Время использовать наши тестовые выборки и сравнить модели на них.

In [22]:
# Поочередно проверим все 3 модели, начнем с модели Дерево решений
tree_predict_test = tree_model.predict(features_test)
accuracy_score(target_test, tree_predict_test)

0.7713841368584758

На тестовой выборке параметр accuracy упал.

In [23]:
# Аналогично проверим Случайный лес
forest_predict_test = forest_model.predict(features_test)
accuracy_score(target_test, forest_predict_test)

0.8009331259720062

И тут показатель accuracy становится меньше

In [24]:
# Проверим Логистическую регрессию
logistic_predict_test = logistic_model.predict(features_test)
accuracy_score(target_test, logistic_predict_test)

0.6967340590979783

Аналогичным предыдущим показателям accuracy падает.

## Вывод по пункту 4:

При проверке моделей на тестовой выборке показатели accuracy составили:

    - Дерево решений 0.7713841368584758
    
    - Случайный лес 0.8009331259720062
    
    - Логистическая регрессия 0.6967340590979783
    
Нашей задачей изначально было найти модель с уровнем accuracy более 0.75, а значит нам подходит как Дерево решений, так и Случайный лес. Как известно из теории - качество оценку у модели Случайный лес выше чем у Дерева решений, но при этом скорость использования напротив у Случайного леса меньше. 

Если работать с большим массивом данных, при этом делая акцент на скорости - больше подходит модель Дерево решений. 
Но, для более качественных и верных результатов, рекомендуется использовать модель Случайный лес.

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, все верно. Тебе удалось достичь хорошего качества!
</div>

<a id='id_5'></a>
## 5. (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность сравним её с моделью DummyClassifier

In [25]:
# импортируем модель
from sklearn.dummy import DummyClassifier

In [26]:
# поработаем с параметром strategy
# начнем с по умолчанию, т.е. stratified
dummy_model = DummyClassifier(random_state=123)
dummy_model.fit(features_train, target_train)
dummy_predict=dummy_model.predict(features_valid)
accuracy_score(target_valid, dummy_predict)

0.5769828926905132

In [27]:
# теперь попробуем “most_frequent”
dummy_model = DummyClassifier(strategy='most_frequent', random_state=123)
dummy_model.fit(features_train, target_train)
dummy_predict=dummy_model.predict(features_valid)
accuracy_score(target_valid, dummy_predict)

0.6982892690513219

In [28]:
# prior
dummy_model = DummyClassifier(strategy='prior', random_state=123)
dummy_model.fit(features_train, target_train)
dummy_predict=dummy_model.predict(features_valid)
accuracy_score(target_valid, dummy_predict)

0.6982892690513219

In [29]:
# uniform
dummy_model = DummyClassifier(strategy='uniform', random_state=123)
dummy_model.fit(features_train, target_train)
dummy_predict=dummy_model.predict(features_valid)
accuracy_score(target_valid, dummy_predict)

0.48522550544323484

Наибольшей показатель при strategy='prior', остановимся на нем.
Время перейти к тестовой выборке и сравнению результатов

In [30]:
dummy_model=DummyClassifier(strategy='prior', random_state=123)
dummy_model.fit(features_train, target_train)
dummy_predict=dummy_model.predict(features_test)
accuracy_score(target_test, dummy_predict)

0.6936236391912908

## Вывод по пункту 5:
Максимальный показатель модели DummyClassifier - 0.6936236391912908. 
Этот показатель ниже чем у всех предыдущих моделей, а значит проверку на адекватность мы прошли успешно!

<div class="alert alert-block alert-success">
<b>Успех:</b> все верно, ты хорошо справился с дополнительным заданием.
</div>

<a id='id_6'></a>
# Общий вывод:

Наиболее подходящей для нас моделью является "Случайный лес", она же 'RandomForestClassifier' (правильность ответов на тестовой выборке составило 80%). Рекомендуется использовать её.

При возникновении проблем со скоростью обработки так же можно использовать модель "Дерево решений" (правильность ответов на тестовой выборке - 77%), она же 'DecisionTreeClassifier', но в таком случае правильность ответов упадет (примерно на 3%).

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец! Отличо проделанная работа. Итоговый вывод хорошо ее описывает, все логично и структурировано.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
